In [1]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=d5c4ea10c91b0ccfe6994ada7a7590295cd5ec45d2fa8cf6ebf245b6c9cc7b98
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [5]:
import logging
import pandas as pd
import nltk
import tqdm
import json
import datasets
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from datasets import Dataset
from itertools import chain
from tensorflow import keras

In [6]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 1e-6  # Learning-rate for training our model
# LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

In [8]:
data = []

with open("/kaggle/input/indosum/indosum/train.01.jsonl", 'r') as f:
    json_list = list(f)
    for json_str in json_list:
        d = json.loads(json_str)
        data.append(d)

In [9]:
len(data) 
# Output : 14262

14262

In [10]:
data_2 = []
for x in range(1, 6):
    with open(f"/kaggle/input/indosum/indosum/train.0{str(x)}.jsonl", 'r') as f:
        json_list_2 = list(f)
        for json_str_2 in json_list_2:
            a = json.loads(json_str_2)
            data_2.append(a)

In [11]:
len(data_2)

71353

In [12]:
data[0].keys(), len(data)

(dict_keys(['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary']),
 14262)

In [13]:
data_category = []

for x in range(0, 5):
    data_category.append(data_2[x]["category"])
data_category

['tajuk utama', 'teknologi', 'hiburan', 'tajuk utama', 'tajuk utama']

In [14]:
def paragraphs_to_text(raw_paragraph_list):
    new_paragraph_list = []
    for i, paragraph in enumerate(raw_paragraph_list):
        paragraph_list = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            paragraph_list.append(sentence)
        
        new_paragraph = ' '.join(paragraph_list)
        new_paragraph_list.append(new_paragraph)

    paragraph_str = ' '.join(new_paragraph_list)
    return paragraph_str

paragraphs_to_text(data[0]['paragraphs'])

'Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari . Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat . " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit . ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah . Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) . Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh . Dia juga tak tahu penyakit apa yang diderita Ry

In [15]:
paragraphs_list = []

# Menggunakan data_2 
for x in range(0,len(data)):
    paragraphs_list.append(paragraphs_to_text(data[x]["paragraphs"]))
paragraphs_list[:5]

['Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari . Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat . " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit . ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah . Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) . Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh . Dia juga tak tahu penyakit apa yang diderita R

In [16]:
def summary_to_text(summary):
    new_summary = []
    for x in summary:
        new_summary.append(' '.join(x))
    hasil = ' '.join(new_summary)
    return hasil

summary_to_text(data[0]["summary"])

'Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .'

In [17]:
# summary_list = []
# summary = [summary_list.append(summary_to_text(data[x]["summary"])) for x in range(0, len(data))]
# summary_list[:5]

summary_list = []

for x in range(0, len(data)):
    summary_list.append(summary_to_text(data[x]["summary"]))
summary_list[:5]

['Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .',
 'Asus memperkenalkan \xa0 ZenFone generasi keempat dan keduanya sama-sama dibekali setup kamera ganda di depan . Mereka adalah Asus ZenFone 4 Selfie Pro ZD552KL dan ZenFone 4 Selfie ZD553KL . Dua ZenFone Selfie anyar ini kabarnya diracik sedemikian rupa sebagai jawaban atas kekurangan yang ada di perangkat - perangkat kompetitor , khususnya pada aspek jangkauan lensa dan performa di kondisi low-light .',
 'Dinas Pariwisata Provinsi Bengkulu kembali menggelar kegiatan Bimbingan Teknis ( Bimtek ) SDM Kepariwisataan dalam menyongson " Visit 2020 Wonderful Bengkulu " pada 8 - 10 November 2017 yang lalu . Kegiatan 

In [18]:
data = pd.DataFrame({
    "text":paragraphs_list,
    "summary":summary_list
})
data

,text,summary
0,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...
2,"Jakarta , CNN Indonesia - - Dinas Pariwisata P...",Dinas Pariwisata Provinsi Bengkulu kembali men...
3,Merdeka.com - Indonesia Corruption Watch ( ICW...,Indonesia Corruption Watch ( ICW ) meminta Kom...
4,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,Jokowi memimpin upacara penurunan bendera . Us...
...,...,...
14257,"Jakarta , CNN Indonesia - - Amerika Serikat di...",Amerika Serikat dilaporkan telah mengirimkan s...
14258,"Bandung , CNN Indonesia - - Borneo FC berhasil...",Borneo FC menahan imbang Persib Bandung pada l...
14259,JAKARTA ( Pos Kota ) – Komisi Pemberantasan Ko...,Komisi Pemberantasan Korupsi ( KPK ) sudah mel...
14260,Merdeka.com - Sebuah kabar gembira datang bagi...,Kabar gembira datang bagi sepakbola Indone...


In [19]:
train_list, test_list = train_test_split(data, test_size=0.1)
print(f"Jumlah train {len(train_list)}", f"Jumlah test {len(test_list)}")

Jumlah train 12835 Jumlah test 1427


In [20]:
train = dict(train_list)
test = dict(test_list)

In [21]:
# Ubah kedalam tipe DatasetDict()
train = Dataset.from_dict(train)
test = Dataset.from_dict(test)
data = datasets.DatasetDict({"train":train, "test":test})
data

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 12835
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 1427
    })
})

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [24]:
# tokenizer("Hey kamu lagi ngapain?") 
tokenizer(["Kamu kemarin kemana aja?", "Kenapa tidak datang tadi malam?"])

{'input_ids': [[8329, 76, 3, 1050, 12181, 3, 1050, 348, 9, 3, 9, 1191, 58, 1], [5704, 9750, 3, 17, 23, 26, 1639, 331, 1725, 3, 17, 9, 26, 23, 1460, 265, 58, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [19]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [20]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [21]:
tokenized_datasets = data.map(preprocess_function, batched=True)
tokenized_datasets

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12835
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1427
    })
})

In [22]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.



All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.

If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [23]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
data_collator

DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<ext

In [24]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(124)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [25]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [26]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [27]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# For now we will use our test set as our validation_data
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

1604/1604 [==============================] - 921s 548ms/step - loss: 1.3208 - val_loss: 0.7854 - RougeL: 0.1693


In [35]:
kalimat_prediksi = "Matahari bersinar terik di Lampung. Sinarnya terhalang rimbunnya pepohonan, sehingga hanya menyisakan berkas tipis. Burung-burung berkicau seolah sedang menyanyikan lagu untuk alam. Bunyi riak jernih sungai beradu dengan batu kali berpadu dengan sahutan dari beberapa penghuni hutan yang lainnya. Ya, inilah tempat tinggal Bora, si anak gajah Lampung yang sekarang tengah asyik bermain bersama teman-temannya di sebuah sungai."

In [36]:
kalimat_prediksi

'Matahari bersinar terik di Lampung. Sinarnya terhalang rimbunnya pepohonan, sehingga hanya menyisakan berkas tipis. Burung-burung berkicau seolah sedang menyanyikan lagu untuk alam. Bunyi riak jernih sungai beradu dengan batu kali berpadu dengan sahutan dari beberapa penghuni hutan yang lainnya. Ya, inilah tempat tinggal Bora, si anak gajah Lampung yang sekarang tengah asyik bermain bersama teman-temannya di sebuah sungai.'

In [37]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    kalimat_prediksi,
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

[{'summary_text': 'berkicau seolah sedang menyanyikan lagu untuk alam . Bunyi riak jernih sungai beradu diegan batu kali berpadu dengan sahutan dari beberapa penghuni hutan yang lainnya .'}]

In [29]:
data["test"][10]["text"]

'Pengembang perangkat akuntansi online Jurnal.id baru-baru ini memperkenalkan fitur barunya yang diberi nama Jurnal Pay . Fitur ini diperuntukkan untuk membantu pengusaha UKM melakukan pembayaran akun melalui virtual account , baik berupa bank transfer ataupun kartu kredit . Sehingga kini para UKM dapat memproses penagihan agar pelanggan mereka melakukan pembayaran online dari setiap invoice yang dikirim . Selain itu , melalui Jurnal Pay , pelaku UKM dapat mengelola dan merangkai rencana keuangan dengan memberikan pilihan proses pembayaran serta menerima pembayaran dengan cepat , di mana pun dan kapan pun . Pengembangan Jurnal Pay sendiri dilakukan bekerja sama dengan Xendit , yang merupakan penyedia jasa infrastruktur teknologi finansial di Asia Tenggara . Xendit menyediakan Payment Gateway Portal untuk pemrosesan transaksi melalui virtual account tadi . Melalui fitur ini , pengguna Jurnal.id tidak perlu repot lagi untuk mendaftarkan virtual account sendiri . Dengan memasukkan beberap

In [87]:
# model.push_to_hub("transformers-qa", organization="keras-io")
# tokenizer.push_to_hub("transformers-qa", organization="keras-io")

In [88]:
# from transformers import TFAutoModelForSeq2SeqLM

# model = TFAutoModelForSeq2SeqLM.from_pretrained("your-username/my-awesome-model"

In [38]:
# model.save_weights("model_summary_ind.h5")

In [39]:
model = model.load_weights('model_summary_ind.h5')

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    data["test"][0]["text"],
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)